# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
0.983033,"""../../../../da…","""11671200884189…","""91640726718368…","""13211218634510…","""11677416372389…",false
0.144221,"""../../../../da…","""91294119658649…","""72034408193519…","""13211218634510…","""20573202545895…",true
0.679745,"""../../../../da…","""11376928002966…","""17187169143002…","""12242721041436…","""11677416372389…",false
1.0,"""../../../../da…","""91294119658649…","""17384847226213…","""24434699689787…","""11677416372389…",false
0.718982,"""../../../../da…","""11671200884189…","""76217144762438…","""24434699689787…","""33972047121872…",true
0.0,"""../../../../da…","""11376928002966…","""13237975058790…","""11201485653184…",null,false
0.02333,"""../../../../da…","""11376928002966…","""15221266732673…","""24434699689787…","""33972047121872…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""../../../../da…","""87211407765889…","""15636757365444…","""FirstMortgage3…","""31994863079766…","""Rejected"""
0.144221,"""../../../../da…","""14827426443313…","""16061509756808…","""FirstMortgage3…","""10882223958817…","""Accepted"""
0.679745,"""../../../../da…","""60216918437190…","""12758509480611…","""MoneyMarketSav…","""31994863079766…","""Rejected"""
1.0,"""../../../../da…","""14827426443313…","""33302119832023…","""BasicChecking""","""31994863079766…","""Rejected"""
0.718982,"""../../../../da…","""87211407765889…","""18422447300973…","""BasicChecking""","""95609170324871…","""Accepted"""
0.0,"""../../../../da…","""60216918437190…","""14092580028975…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""../../../../da…","""60216918437190…","""16501551279156…","""BasicChecking""","""95609170324871…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""../../../../da…","""70351932222699…","""58333950518766…","""FirstMortgage3…","""42355577160528…","""Rejected"""
0.144221,"""../../../../da…","""14398682168795…","""13406412452339…","""FirstMortgage3…","""12298051147893…","""Accepted"""
0.679745,"""../../../../da…","""17032659558441…","""16739956289374…","""MoneyMarketSav…","""42355577160528…","""Rejected"""
1.0,"""../../../../da…","""14398682168795…","""11730224244444…","""BasicChecking""","""42355577160528…","""Rejected"""
0.718982,"""../../../../da…","""70351932222699…","""74758509249330…","""BasicChecking""","""12354541975117…","""Accepted"""
0.0,"""../../../../da…","""17032659558441…","""65264461175144…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""../../../../da…","""17032659558441…","""10670839543086…","""BasicChecking""","""12354541975117…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""61615498372082…","""18243174417114…","""FirstMortgage3…","""75514094785616…","""Rejected"""
0.144221,"""28594643936316…","""13714759292447…","""FirstMortgage3…","""51107376122753…","""Accepted"""
0.679745,"""77108199206847…","""15535964875617…","""MoneyMarketSav…","""75514094785616…","""Rejected"""
1.0,"""28594643936316…","""16793151251414…","""BasicChecking""","""75514094785616…","""Rejected"""
0.718982,"""61615498372082…","""20402169895328…","""BasicChecking""","""10206987716541…","""Accepted"""
0.0,"""77108199206847…","""10232547140510…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""77108199206847…","""54283047069528…","""BasicChecking""","""10206987716541…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""../../../../da…","""af486bc1815c72…","""c73023dda51de1…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
0.144221,"""../../../../da…","""3cdf7b1ae1342e…","""5414b197fb87b3…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
0.679745,"""../../../../da…","""806ef34eb8237f…","""892f5d61a9215e…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
1.0,"""../../../../da…","""3cdf7b1ae1342e…","""c3aa807d9e927a…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
0.718982,"""../../../../da…","""af486bc1815c72…","""c2d492f9a05073…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
0.0,"""../../../../da…","""806ef34eb8237f…","""b21059221605c4…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""../../../../da…","""806ef34eb8237f…","""80cbe489dcecd4…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
